In [1]:
# setup + import
from utils import *
import os
import sklearn
import pandas as pd
import pickle

# setup + import
data = pd.read_csv('../data_processed/loan-10k_lrn.csv')
data.head()

pickle_file = open('../data_processed/loan_column_types.pkl', 'rb')
feature_structure = pickle.load(pickle_file)
pickle_file.close()

In [2]:
feature_columns = feature_structure['bin'] + feature_structure['cat'] + feature_structure['cont'] + feature_structure['ord']

feature_structure["cat"]
TARGET = feature_structure["target"]


X = data[feature_columns]
y = data[TARGET]

values = [v  for _,v in feature_structure.items()]
# pd.DataFrame({"type":[feature_structure.keys()], "columns":  values} )
pd.DataFrame({"type":feature_structure.keys(), "columns":  values} )

,type,columns
0,bin,"[term, pymnt_plan, initial_list_status, applic..."
1,cat,"[home_ownership, verification_status, loan_sta..."
2,ord,"[emp_length, issue_d_month, issue_d_year, earl..."
3,cont,"[loan_amnt, funded_amnt, funded_amnt_inv, int_..."
4,target,grade


In [3]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

classifiers0 = [
    SVC(kernel='linear', random_state=42, decision_function_shape='ovo', probability=True),
    KNeighborsClassifier(n_neighbors=3, n_jobs=-1),
    DecisionTreeClassifier(random_state=42),
]

classifier1 = [ KNeighborsClassifier(n_jobs=-1)]
model1 = get_pipeline(feature_structure,classifier1[0])

# single model training
res_holdout, model_holdout = perform_holdout(X, y, model1, 42)
res_cv, model_cv = perform_cv(X, y, model1)

results = append_results({}, model_holdout, model_cv, res_holdout, res_cv)
pd.DataFrame(results)

c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

,model,accuracy,precision,recall,f1-score,timing
0,KNeighborsClassifier(n_jobs=-1)_Holdout,0.4475,0.423941,0.4475,0.426923,0.192472
1,KNeighborsClassifier(n_jobs=-1)_CV,0.4494,0.437169,0.4494,0.432662,0.455743


Evaluating multiple models

In [4]:
evaluate_models(data, feature_structure, classifiers0)

c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

,model,accuracy,precision,recall,f1-score,timing
0,"SVC(decision_function_shape='ovo', kernel='lin...",0.8615,0.859849,0.8615,0.860237,7.064133
1,"SVC(decision_function_shape='ovo', kernel='lin...",0.8677,0.866371,0.8677,0.866245,36.516361
2,"KNeighborsClassifier(n_jobs=-1, n_neighbors=3)...",0.4150,0.399450,0.4150,0.398232,0.059811
3,"KNeighborsClassifier(n_jobs=-1, n_neighbors=3)_CV",0.4140,0.402942,0.4140,0.398365,0.457776
4,DecisionTreeClassifier(random_state=42)_Holdout,0.9095,0.909675,0.9095,0.909373,0.203491
5,DecisionTreeClassifier(random_state=42)_CV,0.9173,0.917378,0.9173,0.917198,1.102017


Comparison of SVC over kernel functions:

In [5]:

classifiersSVC = [
    SVC(kernel='linear', random_state=42),
    SVC(kernel='rbf', random_state=42),
    SVC(kernel='poly', random_state=42),
    SVC(kernel='sigmoid', random_state=42),
]

resultsSVC = evaluate_models(data, feature_structure, classifiersSVC)
resultsSVC

c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

,model,accuracy,precision,recall,f1-score,timing
0,"SVC(kernel='linear', random_state=42)_Holdout",0.8615,0.859849,0.8615,0.860237,1.610665
1,"SVC(kernel='linear', random_state=42)_CV",0.8677,0.866371,0.8677,0.866245,8.403517
2,SVC(random_state=42)_Holdout,0.8060,0.795141,0.8060,0.797797,3.551466
3,SVC(random_state=42)_CV,0.8210,0.806102,0.8210,0.811697,18.479526
4,"SVC(kernel='poly', random_state=42)_Holdout",0.7050,0.704415,0.7050,0.691538,3.447777
5,"SVC(kernel='poly', random_state=42)_CV",0.7031,0.701652,0.7031,0.689743,16.566060
6,"SVC(kernel='sigmoid', random_state=42)_Holdout",0.6080,0.603239,0.6080,0.604897,1.876983
7,"SVC(kernel='sigmoid', random_state=42)_CV",0.6015,0.595658,0.6015,0.596095,9.502610


Comparison of KNeighbours over k 

In [6]:
classifierKneighbors = [ KNeighborsClassifier(n_jobs=-1, n_neighbors=k) for k in range(2, 13)]
resultsKNeighbors = evaluate_models(data, feature_structure, classifierKneighbors)
resultsKNeighbors

c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

,model,accuracy,precision,recall,f1-score,timing
0,"KNeighborsClassifier(n_jobs=-1, n_neighbors=2)...",0.4235,0.405539,0.4235,0.395926,0.055851
1,"KNeighborsClassifier(n_jobs=-1, n_neighbors=2)_CV",0.4239,0.406861,0.4239,0.397121,0.432842
2,"KNeighborsClassifier(n_jobs=-1, n_neighbors=3)...",0.4150,0.399450,0.4150,0.398232,0.064828
3,"KNeighborsClassifier(n_jobs=-1, n_neighbors=3)_CV",0.4140,0.402942,0.4140,0.398365,0.469743
4,"KNeighborsClassifier(n_jobs=-1, n_neighbors=4)...",0.4420,0.421523,0.4420,0.424452,0.076794
5,"KNeighborsClassifier(n_jobs=-1, n_neighbors=4)_CV",0.4509,0.434156,0.4509,0.434705,0.460768
6,KNeighborsClassifier(n_jobs=-1)_Holdout,0.4475,0.423941,0.4475,0.426923,0.068816
7,KNeighborsClassifier(n_jobs=-1)_CV,0.4494,0.437169,0.4494,0.432662,0.465753
8,"KNeighborsClassifier(n_jobs=-1, n_neighbors=6)...",0.4520,0.431456,0.4520,0.431840,0.072806
9,"KNeighborsClassifier(n_jobs=-1, n_neighbors=6)_CV",0.4504,0.439397,0.4504,0.432855,0.473770


Decision Tree comparison over max_deph

In [7]:
classifierDecisionTree = [ DecisionTreeClassifier(random_state=42, max_depth=depth) for depth in range(3, 15)]
resultsDecisionTree = evaluate_models(data, feature_structure, classifierDecisionTree)
resultsDecisionTree

c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anwender\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av

,model,accuracy,precision,recall,f1-score,timing
0,"DecisionTreeClassifier(max_depth=3, random_sta...",0.8540,0.838185,0.8540,0.842883,0.096738
1,"DecisionTreeClassifier(max_depth=3, random_sta...",0.8682,0.851223,0.8682,0.858037,0.591205
2,"DecisionTreeClassifier(max_depth=4, random_sta...",0.8790,0.871055,0.8790,0.873129,0.117685
3,"DecisionTreeClassifier(max_depth=4, random_sta...",0.8860,0.882210,0.8860,0.880588,0.711098
4,"DecisionTreeClassifier(max_depth=5, random_sta...",0.8915,0.894481,0.8915,0.888021,0.138628
5,"DecisionTreeClassifier(max_depth=5, random_sta...",0.8946,0.897858,0.8946,0.890486,0.777948
6,"DecisionTreeClassifier(max_depth=6, random_sta...",0.9145,0.916999,0.9145,0.912210,0.140624
7,"DecisionTreeClassifier(max_depth=6, random_sta...",0.9099,0.913512,0.9099,0.908944,0.829751
8,"DecisionTreeClassifier(max_depth=7, random_sta...",0.9220,0.922221,0.9220,0.921425,0.155585
9,"DecisionTreeClassifier(max_depth=7, random_sta...",0.9198,0.920117,0.9198,0.918852,0.877681
